In [3]:
import streamlit as st
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import json
import os
import s3fs
from datetime import datetime
import base64
import subprocess


# Enter file paths
NHPD = "/Users/anushreechaudhuri/pCloud Drive/MIT/MIT Work/DC DOE/app_files/equity-tool/data/report/nhpd.geojson"
DAC = "/Users/anushreechaudhuri/pCloud Drive/MIT/MIT Work/DC DOE/app_files/equity-tool/data/report/dac.geojson"
TT = "/Users/anushreechaudhuri/pCloud Drive/MIT/MIT Work/DC DOE/app_files/equity-tool/data/report/tt_shp.geojson"
COUNTIES = "/Users/anushreechaudhuri/pCloud Drive/MIT/MIT Work/DC DOE/app_files/equity-tool/data/report/counties.geojson"
STATES = "/Users/anushreechaudhuri/pCloud Drive/MIT/MIT Work/DC DOE/app_files/equity-tool/data/report/states.geojson"

level = "County"

def load_nhpd():
    return gpd.read_file(NHPD)

def load_dac():
    return gpd.read_file(DAC)

def load_boundary(level):
    if level == "Census Tract ID":
        dac_boundary = load_dac()
        dac_boundary["NAME"] = dac_boundary["GEOID"]
        return dac_boundary
    elif level == "County":
        return gpd.read_file(COUNTIES)
    elif level == "State":
        return gpd.read_file(STATES)
    else:
        return gpd.read_file(TT)


nhpd = load_nhpd()
dac = load_dac()
boundary = load_boundary(level)
print("Done loading!")

Done loading!


In [10]:
import pickle

with open("stuff.pkl", "wb") as f:
    pickle.dump((nhpd, dac, boundary, ), f)

In [1]:
import pickle 
with open("stuff.pkl", "rb") as f:
    nhpd, dac, boundary = pickle.load(f)



In [59]:
import geopandas as gpd 
import plotly.express as px 
import json 

dac = dac.to_crs(boundary.crs)
nhpd = nhpd.to_crs(boundary.crs)

selected = "San Diego County, CA"
level = "County"


    # Find the row in counties that corresponds to selected "county"
shape = boundary.loc[boundary["NAME"] == selected]


# Spatial join of nhpd and boundary shape
nhpd_select = gpd.sjoin(shape, nhpd, how="inner", predicate="intersects")
# Spatial join of dac and boundary shape
def dac_selector(shape, level):
    if level == "Census Tract ID":
        return shape
    if level == "County":
        # Find first five characters of GEOID in dacs
        # return dac.loc[dac["GEOID"].str[:5].equals(shape["GEOID"].values[0][:5])]
        return dac.drop(
            dac[dac["GEOID"].str[:5].ne(shape["GEOID"].values[0])].index
        )
    if level == "State":
        return dac.drop(
            dac[dac["GEOID"].str[:2].ne(shape["GEOID"].values[0])].index
        )
    else:
        return dac.drop(dac[dac["GEOID"].str[:].ne(shape["GEOID"].values[0])].index)

dac_select = dac_selector(shape, level)
if nhpd_select.empty:
    print("No housing data found for this location.")
if dac_select.empty:
    print("No census tracts found for this location.")

    # fig.update_geos(fitbounds="locations")
    # fig.show()    


In [87]:
nhpd_select.lat

2573    33.327346
2573    33.372089
2573    33.372936
2573    33.374664
2573    33.375820
          ...    
2573    32.585334
2573    32.585609
2573    32.585662
2573    32.586296
2573    32.586654
Name: lat, Length: 679, dtype: float64

In [82]:
fig = px.scatter_mapbox(
            nhpd_select,
            lat="lat",
            lon="lon",
            hover_data=["Property Name", "Street Address"],
            mapbox_style="carto-positron",
        )
fig.show()

In [16]:
import pyproj
import pandas as pd 

df = dac_select.to_crs(pyproj.CRS.from_epsg(4326), inplace=False)
df = df[~pd.isna(df.geometry)]
# fig = px.choropleth(df, locations=df.index, color="DAC_status")
# fig.update_geos(fitbounds="locations", visible=False)
# fig.show()
# df.plot()
# fig = px.choropleth_mapbox(df, geojson=df.geometry, locations=df.index)

In [17]:
colors = ['red' if row.DAC_status == "Disadvantaged" else "black" for _, row in df.iterrows()]
stroke_width  = [2 if row.DAC_status == "Disadvantaged" else 0 for _, row in df.iterrows()]

In [18]:
import numpy as np

def get_plotting_zoom_level_and_center_coordinates_from_lonlat_tuples(longitudes=None, latitudes=None):
    """Function documentation:\n
    Basic framework adopted from Krichardson under the following thread:
    https://community.plotly.com/t/dynamic-zoom-for-mapbox/32658/7

    # NOTE:
    # THIS IS A TEMPORARY SOLUTION UNTIL THE DASH TEAM IMPLEMENTS DYNAMIC ZOOM
    # in their plotly-functions associated with mapbox, such as go.Densitymapbox() etc.

    Returns the appropriate zoom-level for these plotly-mapbox-graphics along with
    the center coordinate tuple of all provided coordinate tuples.
    """
    latitudes = np.array(latitudes)
    longitudes = np.array(longitudes)
    
    # Check whether both latitudes and longitudes have been passed,
    # or if the list lenghts don't match
    if ((latitudes is None or longitudes is None)
            or (len(latitudes) != len(longitudes))):
        # Otherwise, return the default values of 0 zoom and the coordinate origin as center point
        return 0, (0, 0)

    # Get the boundary-box 
    b_box = {} 
    b_box['height'] = latitudes.max()-latitudes.min()
    b_box['width'] = longitudes.max()-longitudes.min()
    b_box['center']= (np.mean(longitudes), np.mean(latitudes))

    # get the area of the bounding box in order to calculate a zoom-level
    area = b_box['height'] * b_box['width']

    # * 1D-linear interpolation with numpy:
    # - Pass the area as the only x-value and not as a list, in order to return a scalar as well
    # - The x-points "xp" should be in parts in comparable order of magnitude of the given area
    # - The zpom-levels are adapted to the areas, i.e. start with the smallest area possible of 0
    # which leads to the highest possible zoom value 20, and so forth decreasing with increasing areas
    # as these variables are antiproportional
    zoom = np.interp(x=area,
                     xp=[0, 5**-10, 4**-10, 3**-10, 2**-10, 1**-10, 1**-5],
                     fp=[20, 15,    14,     13,     12,     7,      5])

    # Finally, return the zoom level and the associated boundary-box center coordinates
    return zoom, b_box['center']


In [52]:
def zoom_center(lons: tuple=None, lats: tuple=None, lonlats: tuple=None,
        format: str='lonlat', projection: str='mercator',
        width_to_height: float=2.0):
    """Finds optimal zoom and centering for a plotly mapbox.
    Must be passed (lons & lats) or lonlats.
    Temporary solution awaiting official implementation, see:
    https://github.com/plotly/plotly.js/issues/3434
    
    Parameters
    --------
    lons: tuple, optional, longitude component of each location
    lats: tuple, optional, latitude component of each location
    lonlats: tuple, optional, gps locations
    format: str, specifying the order of longitud and latitude dimensions,
        expected values: 'lonlat' or 'latlon', only used if passed lonlats
    projection: str, only accepting 'mercator' at the moment,
        raises `NotImplementedError` if other is passed
    width_to_height: float, expected ratio of final graph's with to height,
        used to select the constrained axis.
    
    Returns
    --------
    zoom: float, from 1 to 20
    center: dict, gps position with 'lon' and 'lat' keys

    >>> print(zoom_center((-109.031387, -103.385460),
    ...     (25.587101, 31.784620)))
    (5.75, {'lon': -106.208423, 'lat': 28.685861})
    """
    if lons is None and lats is None:
        if isinstance(lonlats, tuple):
            lons, lats = zip(*lonlats)
        else:
            raise ValueError(
                'Must pass lons & lats or lonlats'
            )
    
    maxlon, minlon = max(lons), min(lons)
    maxlat, minlat = max(lats), min(lats)
    center = {
        'lon': round((maxlon + minlon) / 2, 6),
        'lat': round((maxlat + minlat) / 2, 6)
    }
    
    # longitudinal range by zoom level (20 to 1)
    # in degrees, if centered at equator
    lon_zoom_range = np.array([
        0.0007, 0.0014, 0.003, 0.006, 0.012, 0.024, 0.048, 0.096,
        0.192, 0.3712, 0.768, 1.536, 3.072, 6.144, 11.8784, 23.7568,
        47.5136, 98.304, 190.0544, 360.0
    ])
    
    if projection == 'mercator':
        margin = 1.2
        height = (maxlat - minlat) * margin * width_to_height
        width = (maxlon - minlon) * margin
        lon_zoom = np.interp(width , lon_zoom_range, range(20, 0, -1))
        lat_zoom = np.interp(height, lon_zoom_range, range(20, 0, -1))
        zoom = round(min(lon_zoom, lat_zoom), 2)
    else:
        raise NotImplementedError(
            f'{projection} projection is not implemented'
        )
    
    return zoom, center

In [46]:
hagu_center = json.loads(shape.to_crs(pyproj.CRS.from_epsg(4326), inplace=False).centroid.to_json())["features"][0]["geometry"]["coordinates"]

/var/folders/vm/23_sdlnx1sn5ykz3bd8knfbm0000gn/T/ipykernel_28940/641247433.py:1: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [70]:
zoom, center = zoom_center(lats=[i[0] for i in coords[0]], lons=[i[1] for i in coords[0]])

print(zoom)
print(center)

7.82
{'lon': 33.01973, 'lat': -116.838469}


In [ ]:
s

In [94]:
x = json.loads(shape.geometry.to_json())
coords = x['features'][0]['geometry']['coordinates']
zoom, center = zoom_center(lons=[i[0] for i in coords[0]], lats=[i[1] for i in coords[0]])


fig = px.choropleth_mapbox(df, geojson=df['geometry'], locations=df.index, color="avg_energy_burden_natl_pctile",
                           mapbox_style="carto-positron", zoom=0.9 * zoom, center=center)
fig.update_layout(mapbox={
                "style": "carto-positron",
                "layers": [
                    {
                        "source": json.loads(shape.geometry.to_json()),
                        "type": "line",
                        "color": "gray",
                        "line": {"width": 3},
                        "below": "traces",
                    },
                ],
            })

fig.update_geos(fitbounds="locations", visible=False)
fig.update_traces(marker_line_color=colors, marker_line_width=stroke_width, marker_opacity=0.3)

lats = nhpd_select.lat
lons = nhpd_select.lon 



fig.add_scattermapbox(
    lat = lats,
    lon = lons,
    mode = 'markers+text',
    marker_size=3,
    marker_color="green"
)

In [79]:
nhpd_select.geometry

2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
                              ...                        
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
Name: geometry, Length: 679, dtype: geometry

In [81]:
nhpd_select.geometry

2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
                              ...                        
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
2573    POLYGON ((-117.59600 33.38779, -117.59360 33.3...
Name: geometry, Length: 679, dtype: geometry